#### Pour ce Notebook , on a utilisé le modéle "all-MiniLM-L6-v2" qui est entrainé sur des données textuelles en anglais 
#### On a enlevé les lignes vides de notre database , les stopwords , on a fusionné les 4 colonnes textuelles dans une seule colonne docs
#### Enfin on a utilisé le modéle pour les classer 

In [4]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer


### Utilsier un modéle entrainée sur des données textuelles écrites en anglais ***all_minilm_l6-v2***

In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


c:\Users\AnasJB\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\AnasJB\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Création et entrainement du modéle BERTopic
### Liste des topics 

## 1 

- Colonne docs

In [9]:
import pandas as pd
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP  # Seulement si vous souhaitez personnaliser UMAP

# ----- ÉTAPE A & B : Charger et nettoyer vos données -----

# 1) Lecture du fichier Excel
df = pd.read_excel("../../Donnees/fichier_traduit.xlsx")


# Par exemple, vos colonnes textuelles sont 'presentation', 'activites', 'réponse1', 'réponse2'
colonnes_textuelles = ['presentation', 'activites', 'réponse1', 'réponse2']

# 2) Remplacer les NaN par des chaînes vides
df[colonnes_textuelles] = df[colonnes_textuelles].fillna("")

# 3) Supprimer les lignes entièrement vides (ou ne contenant que des espaces)
def est_vide_ou_espace(val):
    return val.strip() == ""

mask_ligne_vide = df[colonnes_textuelles].applymap(est_vide_ou_espace).all(axis=1)
df = df[~mask_ligne_vide].copy()

print("Nombre de lignes restantes après nettoyage :", len(df))

# 4) Créer la liste 'docs' en fusionnant les colonnes textuelles
docs = (
    df["presentation"] + " " +
    df["activites"]    + " " +
    df["réponse1"]     + " " +
    df["réponse2"]
).tolist()

# ----- ÉTAPE C : Entraîner BERTopic -----

# (Optionnel) Personnaliser UMAP si vous rencontrez des problèmes de dimension
umap_model = UMAP(
    n_neighbors=2,
    n_components=2,
    min_dist=0.0,
    metric='cosine'
)

# (Optionnel) Vectorizer pour supprimer les stopwords anglais
vectorizer_model = CountVectorizer(stop_words='english')

# Initialisation du modèle BERTopic
topic_model = BERTopic(
    language="english",                # Ajustez la langue si nécessaire
    vectorizer_model=vectorizer_model, # Retrait des stopwords
    umap_model=umap_model,             # Réduction de dimension personnalisée
    calculate_probabilities=False      # Mettez True si vous voulez les probs (et si votre dataset est assez grand)
)

# Entraînement du modèle
topics, probs = topic_model.fit_transform(docs)

# ----- AFFICHAGE DES TOPICS -----

# 1. Informations générales sur les topics
topic_info = topic_model.get_topic_info()
print("\n=== Aperçu des topics ===")
print(topic_info.head(10))

# 2. Mots-clés du premier topic
keywords_topic0 = topic_model.get_topic(0)
print("\n=== Mots-clés du topic 0 ===")
print(keywords_topic0)

# ----- VISUALISATIONS -----

# 1. Visualisation des topics (carte 2D via UMAP)
fig1 = topic_model.visualize_topics()
fig1.show()

# 2. Visualiser un diagramme en barres des topics
fig2 = topic_model.visualize_barchart(top_n_topics=5, n_words=5)
fig2.show()

# 3. Visualiser la hiérarchie des topics (dendrogramme)
fig3 = topic_model.visualize_hierarchy()
fig3.show()

# 4. Visualiser la matrice de similitude (heatmap) entre topics
fig4 = topic_model.visualize_heatmap()
fig4.show()


Nombre de lignes restantes après nettoyage : 272

=== Aperçu des topics ===
   Topic  Count                                   Name  \
0     -1     59               -1_artists_art_new_space   
1      0     38      0_art_contemporary_artists_artist   
2      1     24               1_work_artists_art_space   
3      2     20   2_artists_projects_space_exhibitions   
4      3     19     3_artists_gallery_place_exhibition   
5      4     17  4_giannozzo_berlin_located_flutgraben   
6      5     16            5_space_wall_project_street   
7      6     15                6_space_art_artists_ifp   
8      7     13    7_x000d_exhibition_solo_association   
9      8     13      8_art_activity_exhibition_artists   

                                      Representation  \
0  [artists, art, new, space, place, work, media,...   
1  [art, contemporary, artists, artist, arts, pro...   
2  [work, artists, art, space, international, exh...   
3  [artists, projects, space, exhibitions, exhibi...   
4  [a